Alternus Vera Sprint 2

Submitted by Harika Nalam
Team Name: The Trio

Main Factor - News Coverage
Micro Factors - Business, Reports, Book Reviews, Broadcast

Problem Statement  - Given an article classifiy the text to a particular. Score is calculated which gives the coverage of the article to a particular topic.    

LDA:   
LDA is used to classify text in a document in aparticular topic using the score, the score the probability distribution of the words.    

DataSet1 - [link](https://components.one/datasets/all-the-news-articles-dataset/)


In [ ]:
import pandas as pd
import sqlite3
import seaborn as sns
from nltk import sent_tokenize

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
db = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/all-the-news.db'

In [ ]:
cnx = sqlite3.connect(db)

In [ ]:
df = pd.read_sql_query("SELECT * FROM longform", cnx)
df.head()

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017,5,Verge,Longform,1.0,None,None
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017,5,Verge,Longform,1.0,None,None
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017,5,Verge,Longform,1.0,None,None
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017,5,Verge,Longform,1.0,None,None
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017,5,Verge,Longform,1.0,None,None


In [ ]:
train = df.sample(frac = 0.7)
test = df.drop(train.index)

In [ ]:
df=train

Tokenization and Lemmatize

Data Cleaning:
1. Tokenization: split the text into and sentences into words. Lowercase the words and remove punctuation
2. Lemmatized: Words in third person are changed to first person and verbs in past and furture tense are changed into present

In [ ]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
df['category'].unique()

array(['newspaper', 'general', 'radio', 'broadcast', None, 'business',
       'wire', 'Reports', 'Exclusive', 'Longform', 'Book Reviews'],
      dtype=object)

group data by category

In [ ]:
data_group = df.groupby('category')
# data_group.head(5)

And the categories I'm considering are 
1. Business
2. Boradcast
3. Book Reviews
4. Reports

In [ ]:
data_business = data_group.get_group('business')
data_broadcast = data_group.get_group('broadcast')
data_BookReviews = data_group.get_group('Book Reviews')
data_Reports = data_group.get_group('Reports')

##Business Data

In [ ]:
data_business = data_business[:300000][['title']]
data_business['index'] = data_business.index
documents_business = data_business

Tokenize the document

In [ ]:
documents_business['title'][56128]

"Trump focuses New Year's tweet on 'many enemies' and everyone who 'lost so badly'"

In [ ]:
document_num = 56128
doc_sample = documents_business[documents_business['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Trump', 'focuses', 'New', "Year's", 'tweet', 'on', "'many", "enemies'", 'and', 'everyone', 'who', "'lost", 'so', "badly'"]


Tokenized and lemmatized document: 
['trump', 'focus', 'year', 'tweet', 'enemi', 'lose', 'bad']


In [ ]:
documents_business.head()

,title,index
57635,'This is not the end': John McCain warns Trump...,57635
61585,Here’s why the most popular map of the world i...,61585
63596,Obama on Trump: 'Don't be bamboozled — don't r...,63596
63273,Trump says 'every poll' shows he won the secon...,63273
61350,This 747 private jet makes Donald Trump's plan...,61350


In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/BusinessArticles.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  documents_business.to_csv(f)

In [ ]:
processed_docs_business = documents_business['title'].map(preprocess)
processed_docs_business.head()

57635    [john, mccain, warn, trump, torch, rand, paul,...
61585                     [popular, world, total, mislead]
63596                 [obama, trump, bamboozl, okey, doke]
63273    [trump, say, poll, show, second, debat, scient...
61350     [privat, make, donald, trump, plane, look, tini]
Name: title, dtype: object

Bag of words:    
Create a dictionary containing the number of times a word appears in the training set. 
Pass the documents to gensim.corpora.Dictionary()

In [ ]:
dictionary_business = gensim.corpora.Dictionary(processed_docs_business)
count = 0
for k, v in dictionary_business.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 john
1 mccain
2 missil
3 paul
4 rand
5 strike


In [ ]:
import pickle 

with open("/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dictionary_business.pkl", "wb") as pkl_handle:
	pickle.dump(dictionary_business, pkl_handle)

Gensim doc2bow:   
convert document into the bag-of-words formart = list(word_id, word,count) 

In [ ]:
bow_corpus_business = [dictionary_business.doc2bow(doc) for doc in processed_docs_business]

TF-IDF: Term Frequency, Inverse Document Frequency.    
1. TFIDF is used to score the importance of words in a document based on their frequency across multiple documents
2. If words appears frequenctly in a document is important and given high score
3. words appear in many document are not considered unique, low score for sucnh words. like the , for


Example: TF IDF
document d1 - 1000 words and word "stock" appears 5 times
TF = 5/1000 = 0.005

10 million documents word "stock" appears 1000 times then
IDF  = log(10000000/1000)= 4

TF-IDF = 0.005 *4 = 0.02


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus_business)
corpus_tfidf_business = tfidf[bow_corpus_business]
from pprint import pprint
for doc in corpus_tfidf_business:
    pprint(doc)
    break

[(0, 0.27684219828328904),
 (1, 0.343582151323745),
 (2, 0.3242744291429671),
 (3, 0.31256090958696153),
 (4, 0.40448164364776595),
 (5, 0.3028959596572979),
 (6, 0.27547341005251974),
 (7, 0.42042875469298785),
 (8, 0.09348045487243165),
 (9, 0.2927893093285109)]


Running LDA using Bag of words





In [ ]:
lda_model_business = gensim.models.LdaMulticore(bow_corpus_business, 
                                       num_topics=10, 
                                       id2word = dictionary_business, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
for idx, topic in lda_model_business.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.016*"trump" + 0.014*"hous" + 0.011*"appl" + 0.010*"report" + 0.007*"white" + 0.007*"work" + 0.006*"clinton" + 0.006*"busi" + 0.005*"republican" + 0.005*"peopl" 
Words: 0


Topic: 0.014*"tech" + 0.013*"happen" + 0.013*"high" + 0.012*"thing" + 0.012*"school" + 0.011*"class" + 0.010*"coolest" + 0.010*"today" + 0.009*"say" + 0.008*"trump" 
Words: 1


Topic: 0.037*"trump" + 0.015*"donald" + 0.008*"clinton" + 0.007*"make" + 0.007*"hillari" + 0.007*"innov" + 0.006*"impact" + 0.006*"zika" + 0.006*"radiolog" + 0.006*"dermatolog" 
Words: 2


Topic: 0.030*"trump" + 0.010*"go" + 0.010*"best" + 0.010*"googl" + 0.010*"latest" + 0.009*"industri" + 0.009*"thing" + 0.009*"market" + 0.008*"extens" + 0.008*"chrome" 
Words: 3


Topic: 0.020*"year" + 0.017*"trump" + 0.016*"come" + 0.011*"protect" + 0.011*"type" + 0.011*"famili" + 0.011*"insur" + 0.010*"thing" + 0.009*"buy" + 0.009*"billion" 
Words: 4


Topic: 0.011*"trump" + 0.007*"like" + 0.007*"clinton" + 0.006*"want" + 0.006*"biggest" + 0.006*"

Running LDA using TF-IDF

In [ ]:
lda_model_tfidf_business = gensim.models.LdaMulticore(corpus_tfidf_business, 
                                       num_topics=10, 
                                       id2word = dictionary_business, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
for idx, topic in lda_model_tfidf_business.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.013*"year" + 0.011*"famili" + 0.011*"protect" + 0.011*"insur" + 0.011*"type" + 0.010*"come" + 0.008*"like" + 0.006*"look" + 0.006*"industri" + 0.006*"citi"


Topic: 1 Word: 0.008*"extens" + 0.008*"chrome" + 0.008*"latest" + 0.007*"best" + 0.007*"googl" + 0.007*"impact" + 0.007*"headlin" + 0.007*"zika" + 0.007*"news" + 0.007*"radiolog"


Topic: 2 Word: 0.005*"trump" + 0.005*"machin" + 0.005*"explain" + 0.005*"wayback" + 0.005*"archiv" + 0.005*"internet" + 0.004*"fund" + 0.004*"hedg" + 0.004*"startup" + 0.004*"dream"


Topic: 3 Word: 0.015*"thing" + 0.007*"today" + 0.007*"tyson" + 0.007*"degrass" + 0.007*"happen" + 0.007*"neil" + 0.006*"coolest" + 0.006*"school" + 0.006*"help" + 0.006*"class"


Topic: 4 Word: 0.008*"gore" + 0.008*"futur" + 0.008*"planet" + 0.008*"degrass" + 0.008*"tyson" + 0.008*"neil" + 0.006*"trump" + 0.005*"clinton" + 0.004*"know" + 0.004*"debat"


Topic: 5 Word: 0.008*"trump" + 0.004*"buy" + 0.004*"leas" + 0.004*"mind" + 0.004*"donald" + 0.004*"thing

Save Models

In [ ]:
lda_model_tfidf_business.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/LDA_TFID_Business.sav')
lda_model_business.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/lda_model_business.sav')


In [ ]:
unseen_document = "Dow tumbles more than 900 points, Nasdaq drops 4 percent to close out brutal month"

# Data preprocessing step for the unseen document
bow_vector = dictionary_business.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_business.print_topic(index, 5)))

Score: 0.3604547679424286	 Topic: 0.042*"trump" + 0.014*"donald" + 0.009*"clinton" + 0.009*"obama" + 0.008*"hillari"
Score: 0.35863015055656433	 Topic: 0.016*"tyson" + 0.016*"degrass" + 0.016*"neil" + 0.015*"futur" + 0.012*"compani"
Score: 0.20312029123306274	 Topic: 0.037*"trump" + 0.014*"say" + 0.011*"compani" + 0.009*"world" + 0.009*"help"
Score: 0.011114311404526234	 Topic: 0.020*"year" + 0.017*"trump" + 0.016*"come" + 0.011*"protect" + 0.011*"type"
Score: 0.011114184744656086	 Topic: 0.037*"trump" + 0.015*"donald" + 0.008*"clinton" + 0.007*"make" + 0.007*"hillari"
Score: 0.011113964952528477	 Topic: 0.012*"trump" + 0.011*"wall" + 0.011*"street" + 0.009*"like" + 0.008*"explain"
Score: 0.011113610118627548	 Topic: 0.014*"tech" + 0.013*"happen" + 0.013*"high" + 0.012*"thing" + 0.012*"school"
Score: 0.011113563552498817	 Topic: 0.030*"trump" + 0.010*"go" + 0.010*"best" + 0.010*"googl" + 0.010*"latest"
Score: 0.01111323107033968	 Topic: 0.011*"trump" + 0.007*"like" + 0.007*"clinton" + 

##Broadcast

In [ ]:
data_broadcast = data_broadcast[:300000][['title']]
data_broadcast['index'] = data_broadcast.index
documents_broadcast = data_broadcast

In [ ]:
documents_broadcast

,title,index
75433,Judge Napolitano: What happened to the FBI? It...,75433
47106,Cavendish in yellow after first stage win,47106
55824,Chattanooga: Why were our soldiers so vulnerable?,55824
53683,North Korea's first lady Ri Sol Ju seen for fi...,53683
54898,McKinney pool party incident was about race,54898
...,...,...
72697,5 North Carolina deputies disciplined in conne...,72697
54808,8 must-see Budapest buildings,54808
55672,Boater's incredible reaction to whale encounte...,55672
45758,Navy SEAL who claims he killed bin Laden bough...,45758


In [ ]:
documents_broadcast['title'][41840]

'Istanbul attack: Dozens killed at nightclub'

In [ ]:
document_num = 41840
doc_sample = documents_broadcast[documents_broadcast['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Istanbul', 'attack:', 'Dozens', 'killed', 'at', 'nightclub']


Tokenized and lemmatized document: 
['istanbul', 'attack', 'dozen', 'kill', 'nightclub']


In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/BroadcastArticles.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  documents_broadcast.to_csv(f)

In [ ]:
processed_docs_broadcast = documents_broadcast['title'].map(preprocess)
processed_docs_broadcast.head()

75433    [judg, napolitano, happen, corrupt, obama, team]
47106                          [cavendish, yellow, stage]
55824                      [chattanooga, soldier, vulner]
53683               [north, korea, ladi, see, time, year]
54898                [mckinney, pool, parti, incid, race]
Name: title, dtype: object

Bag of words

In [ ]:
dictionary_broadcast = gensim.corpora.Dictionary(processed_docs_broadcast)
count = 0
for k, v in dictionary_broadcast.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 corrupt
1 happen
2 judg
3 napolitano
4 obama
5 team


Gensim doc2bow

In [ ]:
bow_corpus_broadcast = [dictionary_broadcast.doc2bow(doc) for doc in processed_docs_broadcast]

TF-IDF

In [ ]:

tfidf = models.TfidfModel(bow_corpus_broadcast)
corpus_tfidf_broadcast = tfidf[bow_corpus_broadcast]
from pprint import pprint
for doc in corpus_tfidf_broadcast:
    pprint(doc)
    break

[(0, 0.46494242285004256),
 (1, 0.37172365844892646),
 (2, 0.35270308089557617),
 (3, 0.590808249782448),
 (4, 0.2397752127910462),
 (5, 0.33867988016777134)]


Running LDA model using Bag of words

In [ ]:
lda_model_broadcast = gensim.models.LdaMulticore(bow_corpus_broadcast, 
                                       num_topics=10, 
                                       id2word = dictionary_broadcast, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
for idx, topic in lda_model_broadcast.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.027*"trump" + 0.018*"say" + 0.013*"clinton" + 0.006*"hous" + 0.006*"report" + 0.005*"call" + 0.005*"citi" + 0.005*"campaign" + 0.005*"state" + 0.005*"black" 
Words: 0


Topic: 0.022*"trump" + 0.021*"kill" + 0.010*"say" + 0.010*"student" + 0.007*"isi" + 0.007*"news" + 0.006*"shoot" + 0.005*"video" + 0.005*"india" + 0.005*"tell" 
Words: 1


Topic: 0.013*"shoot" + 0.013*"attack" + 0.013*"polic" + 0.011*"dead" + 0.010*"isi" + 0.010*"video" + 0.009*"help" + 0.008*"trump" + 0.008*"death" + 0.007*"kill" 
Words: 2


Topic: 0.035*"trump" + 0.011*"shoot" + 0.010*"video" + 0.010*"clinton" + 0.008*"obama" + 0.007*"polic" + 0.005*"dead" + 0.005*"leav" + 0.005*"call" + 0.005*"kill" 
Words: 3


Topic: 0.039*"trump" + 0.014*"world" + 0.008*"look" + 0.006*"polici" + 0.006*"kill" + 0.006*"penc" + 0.005*"donald" + 0.005*"video" + 0.005*"opinion" + 0.004*"hotel" 
Words: 4


Topic: 0.048*"trump" + 0.015*"clinton" + 0.014*"obama" + 0.010*"korea" + 0.010*"north" + 0.008*"hous" + 0.007*"travel" + 0.0

Running LDA using TF-IDF

In [ ]:
lda_model_tfidf_broadcast = gensim.models.LdaMulticore(corpus_tfidf_broadcast, 
                                       num_topics=10, 
                                       id2word = dictionary_broadcast, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
for idx, topic in lda_model_tfidf_broadcast.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.006*"trump" + 0.006*"hotel" + 0.004*"video" + 0.004*"well" + 0.004*"report" + 0.004*"clinton" + 0.003*"kill" + 0.003*"fargo" + 0.003*"citi" + 0.003*"attack"


Topic: 1 Word: 0.007*"trump" + 0.005*"shoot" + 0.005*"clinton" + 0.004*"hous" + 0.004*"video" + 0.004*"white" + 0.004*"romney" + 0.003*"biden" + 0.003*"america" + 0.003*"week"


Topic: 2 Word: 0.008*"trump" + 0.006*"video" + 0.005*"live" + 0.004*"presid" + 0.004*"happen" + 0.004*"olymp" + 0.004*"fund" + 0.003*"obama" + 0.003*"clinton" + 0.003*"kid"


Topic: 3 Word: 0.007*"trump" + 0.006*"video" + 0.006*"say" + 0.004*"clinton" + 0.004*"opinion" + 0.003*"obama" + 0.003*"come" + 0.003*"climat" + 0.003*"chang" + 0.003*"power"


Topic: 4 Word: 0.007*"trump" + 0.005*"video" + 0.005*"north" + 0.004*"korea" + 0.004*"photo" + 0.004*"week" + 0.003*"hous" + 0.003*"arrest" + 0.003*"death" + 0.003*"immigr"


Topic: 5 Word: 0.010*"clinton" + 0.010*"trump" + 0.005*"fast" + 0.005*"fact" + 0.005*"email" + 0.004*"state" + 0.004*"v

Save models

In [ ]:
lda_model_tfidf_broadcast.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/LDA_TFID_broadcast.sav')
lda_model_broadcast.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/lda_model_broadcast.sav')


In [ ]:
unseen_document = "Country music legend Naomi Judd, one half of The Judds, dies age 76"

# Data preprocessing step for the unseen document
bow_vector = dictionary_broadcast.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_broadcast.print_topic(index, 5)))

Score: 0.704056978225708	 Topic: 0.023*"trump" + 0.022*"fact" + 0.022*"fast" + 0.006*"attack" + 0.006*"time"
Score: 0.19591523706912994	 Topic: 0.048*"trump" + 0.015*"clinton" + 0.014*"obama" + 0.010*"korea" + 0.010*"north"
Score: 0.012510238215327263	 Topic: 0.013*"shoot" + 0.013*"attack" + 0.013*"polic" + 0.011*"dead" + 0.010*"isi"
Score: 0.012504118494689465	 Topic: 0.011*"report" + 0.010*"investig" + 0.009*"clinton" + 0.008*"video" + 0.008*"trump"
Score: 0.01250371988862753	 Topic: 0.027*"trump" + 0.018*"say" + 0.013*"clinton" + 0.006*"hous" + 0.006*"report"
Score: 0.012503646314144135	 Topic: 0.039*"trump" + 0.014*"world" + 0.008*"look" + 0.006*"polici" + 0.006*"kill"
Score: 0.0125028220936656	 Topic: 0.022*"trump" + 0.021*"kill" + 0.010*"say" + 0.010*"student" + 0.007*"isi"
Score: 0.012501639313995838	 Topic: 0.030*"trump" + 0.030*"clinton" + 0.015*"video" + 0.009*"email" + 0.008*"poll"
Score: 0.012501147575676441	 Topic: 0.035*"trump" + 0.011*"shoot" + 0.010*"video" + 0.010*"cli

##Book Reviews

In [ ]:
data_BookReviews = data_BookReviews[:300000][['title']]
data_BookReviews['index'] = data_BookReviews.index
documents_BookReviews = data_BookReviews

In [ ]:
documents_BookReviews['title'][1213]

"Malka Older's Infomocracy is this election season's essential thriller"

In [ ]:
document_num = 1213
doc_sample = documents_BookReviews[documents_BookReviews['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Malka', "Older's", 'Infomocracy', 'is', 'this', 'election', "season's", 'essential', 'thriller']


Tokenized and lemmatized document: 
['malka', 'older', 'infomocraci', 'elect', 'season', 'essenti', 'thriller']


In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/BookReview.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  documents_BookReviews.to_csv(f)

In [ ]:
processed_docs_BookReviews = documents_BookReviews['title'].map(preprocess)
processed_docs_BookReviews.head()

1213    [malka, older, infomocraci, elect, season, ess...
1216               [book, scienc, fiction, page, histori]
1210    [stanley, robinson, york, glorious, think, exp...
1208           [chang, agent, terribl, book, great, movi]
1206    [michael, crichton, novel, dinosaur, bone, gun...
Name: title, dtype: object

Bag of words

In [ ]:
dictionary_BookReviews = gensim.corpora.Dictionary(processed_docs_BookReviews)
count = 0
for k, v in dictionary_BookReviews.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 elect
1 essenti
2 infomocraci
3 malka
4 older
5 season


Gensim doc2bow

In [ ]:
bow_corpus_BookReviews = [dictionary_BookReviews.doc2bow(doc) for doc in processed_docs_BookReviews]

TF - IDF

In [ ]:
tfidf = models.TfidfModel(bow_corpus_BookReviews)
corpus_tfidf_BookReviews = tfidf[bow_corpus_BookReviews]
from pprint import pprint
for doc in corpus_tfidf_BookReviews:
    pprint(doc)
    break

[(0, 0.37796447300922725),
 (1, 0.37796447300922725),
 (2, 0.37796447300922725),
 (3, 0.37796447300922725),
 (4, 0.37796447300922725),
 (5, 0.37796447300922725),
 (6, 0.37796447300922725)]


Running LDA using Bag of words

In [ ]:
lda_model_BookReviews = gensim.models.LdaMulticore(bow_corpus_BookReviews, 
                                       num_topics=10, 
                                       id2word = dictionary_BookReviews, 
                                       passes = 2, 
                                       workers=2)

for idx, topic in lda_model_BookReviews.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.058*"book" + 0.058*"mar" + 0.058*"terribl" + 0.058*"cost" + 0.058*"wander" + 0.058*"human" + 0.058*"movi" + 0.058*"mission" + 0.058*"agent" + 0.058*"great" 
Words: 0


Topic: 0.014*"book" + 0.014*"data" + 0.014*"comic" + 0.014*"chang" + 0.014*"look" + 0.014*"futur" + 0.014*"stupid" + 0.014*"blog" + 0.014*"beauti" + 0.014*"world" 
Words: 1


Topic: 0.014*"book" + 0.014*"chang" + 0.014*"futur" + 0.014*"blog" + 0.014*"captur" + 0.014*"world" + 0.014*"okcupid" + 0.014*"comic" + 0.014*"think" + 0.014*"autom" 
Words: 2


Topic: 0.073*"novel" + 0.073*"lack" + 0.073*"dinosaur" + 0.073*"gunsling" + 0.073*"bone" + 0.073*"crichton" + 0.073*"michael" + 0.073*"soul" + 0.007*"book" + 0.007*"futur" 
Words: 3


Topic: 0.014*"book" + 0.014*"futur" + 0.014*"data" + 0.014*"look" + 0.014*"world" + 0.014*"think" + 0.014*"captur" + 0.014*"comic" + 0.014*"chang" + 0.014*"great" 
Words: 4


Topic: 0.052*"chang" + 0.052*"robinson" + 0.052*"think" + 0.052*"experi" + 0.052*"glorious" + 0.052*"climat" + 

Running LDA using TF IDF

In [ ]:
lda_model_tfidf_BookReviews = gensim.models.LdaMulticore(corpus_tfidf_BookReviews, 
                                       num_topics=10, 
                                       id2word = dictionary_BookReviews, 
                                       passes = 2, 
                                       workers=2)

for idx, topic in lda_model_tfidf_BookReviews.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.046*"path" + 0.046*"obscura" + 0.046*"entir" + 0.046*"guid" + 0.046*"travel" + 0.046*"atlas" + 0.046*"beat" + 0.036*"world" + 0.026*"book" + 0.010*"data" 
Words: 0


Topic: 0.046*"soul" + 0.046*"crichton" + 0.046*"michael" + 0.046*"dinosaur" + 0.046*"lack" + 0.046*"bone" + 0.046*"gunsling" + 0.046*"novel" + 0.010*"book" + 0.010*"okcupid" 
Words: 1


Topic: 0.014*"book" + 0.014*"blog" + 0.014*"data" + 0.014*"fiction" + 0.014*"page" + 0.014*"futur" + 0.014*"william" + 0.014*"okcupid" + 0.014*"stupid" + 0.014*"world" 
Words: 2


Topic: 0.014*"book" + 0.014*"data" + 0.014*"comic" + 0.014*"futur" + 0.014*"peripher" + 0.014*"page" + 0.014*"okcupid" + 0.014*"scienc" + 0.014*"movi" + 0.014*"fiction" 
Words: 3


Topic: 0.044*"thousand" + 0.044*"cultur" + 0.044*"spirit" + 0.044*"captur" + 0.044*"comic" + 0.042*"mar" + 0.042*"cost" + 0.042*"wander" + 0.042*"mission" + 0.042*"human" 
Words: 4


Topic: 0.014*"book" + 0.014*"okcupid" + 0.014*"data" + 0.014*"scienc" + 0.014*"look" + 0.014*"b

In [ ]:
lda_model_tfidf_BookReviews.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/LDA_TFID_bookreview.sav')
lda_model_BookReviews.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/lda_model_bookreview.sav')

In [ ]:

unseen_document = "At 100, the ‘Just William’ Books Are an Icon of British Childhood"

# Data preprocessing step for the unseen document
bow_vector = dictionary_BookReviews.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_BookReviews.print_topic(index, 5)))

Score: 0.6999821662902832	 Topic: 0.064*"book" + 0.064*"futur" + 0.064*"form" + 0.064*"data" + 0.064*"gibson"
Score: 0.03334229812026024	 Topic: 0.091*"book" + 0.091*"page" + 0.091*"fiction" + 0.091*"scienc" + 0.091*"histori"
Score: 0.03333878517150879	 Topic: 0.058*"book" + 0.058*"mar" + 0.058*"terribl" + 0.058*"cost" + 0.058*"wander"
Score: 0.033336758613586426	 Topic: 0.070*"world" + 0.037*"guid" + 0.037*"atlas" + 0.037*"think" + 0.037*"beat"
Score: 0.03333333507180214	 Topic: 0.014*"book" + 0.014*"data" + 0.014*"comic" + 0.014*"chang" + 0.014*"look"
Score: 0.03333333507180214	 Topic: 0.014*"book" + 0.014*"chang" + 0.014*"futur" + 0.014*"blog" + 0.014*"captur"
Score: 0.03333333507180214	 Topic: 0.073*"novel" + 0.073*"lack" + 0.073*"dinosaur" + 0.073*"gunsling" + 0.073*"bone"
Score: 0.03333333507180214	 Topic: 0.014*"book" + 0.014*"futur" + 0.014*"data" + 0.014*"look" + 0.014*"world"
Score: 0.03333333507180214	 Topic: 0.052*"chang" + 0.052*"robinson" + 0.052*"think" + 0.052*"experi" 

##Reprts

In [ ]:

data_Reports = data_Reports[:300000][['title']]
data_Reports['index'] = data_Reports.index
documents_Reports = data_Reports

In [ ]:
documents_Reports['title'][683]

"Samsung's razor-thin Galaxy Tab S takes another run at the iPad"

In [ ]:
document_num = 683
doc_sample = documents_Reports[documents_Reports['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
["Samsung's", 'razor-thin', 'Galaxy', 'Tab', 'S', 'takes', 'another', 'run', 'at', 'the', 'iPad']


Tokenized and lemmatized document: 
['samsung', 'razor', 'galaxi', 'take', 'ipad']


In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/reprtsData.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  documents_Reports.to_csv(f)

In [ ]:
processed_docs_Reports = documents_Reports['title'].map(preprocess)
processed_docs_Reports.head()

693                                  [miss, world, goal]
335               [volvo, thanksgiv, road, test, sweden]
140    [fed, look, experiment, light, field, camera, ...
466    [like, googl, glass, car, tri, mini, augment, ...
372    [lean, muscl, drive, lighter, better, chevi, c...
Name: title, dtype: object

In [ ]:
dictionary_Reports = gensim.corpora.Dictionary(processed_docs_Reports)
count = 0
for k, v in dictionary_Reports.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 goal
1 miss
2 world
3 road
4 sweden
5 test


Gensim doc2bow

In [ ]:
bow_corpus_Reports = [dictionary_Reports.doc2bow(doc) for doc in processed_docs_Reports]

TF IDF

In [ ]:
tfidf = models.TfidfModel(bow_corpus_Reports)
corpus_tfidf_Reports = tfidf[bow_corpus_Reports]
from pprint import pprint
for doc in corpus_tfidf_BookReviews:
    pprint(doc)
    break

[(0, 0.37796447300922725),
 (1, 0.37796447300922725),
 (2, 0.37796447300922725),
 (3, 0.37796447300922725),
 (4, 0.37796447300922725),
 (5, 0.37796447300922725),
 (6, 0.37796447300922725)]


Running LDA using bag of words

In [ ]:
lda_model_Reports = gensim.models.LdaMulticore(bow_corpus_Reports, 
                                       num_topics=10, 
                                       id2word = dictionary_Reports, 
                                       passes = 2, 
                                       workers=2)

for idx, topic in lda_model_Reports.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.010*"futur" + 0.008*"realiti" + 0.008*"anim" + 0.008*"googl" + 0.008*"virtual" + 0.005*"camera" + 0.005*"amazon" + 0.005*"drone" + 0.005*"scientist" + 0.005*"film" 
Words: 0


Topic: 0.014*"facebook" + 0.011*"look" + 0.010*"like" + 0.010*"world" + 0.009*"twitter" + 0.008*"kill" + 0.007*"mobil" + 0.007*"app" + 0.007*"game" + 0.006*"secur" 
Words: 1


Topic: 0.008*"googl" + 0.008*"internet" + 0.008*"game" + 0.006*"high" + 0.006*"help" + 0.006*"human" + 0.006*"know" + 0.006*"instagram" + 0.006*"insid" + 0.004*"want" 
Words: 2


Topic: 0.012*"year" + 0.011*"microsoft" + 0.011*"reveal" + 0.009*"love" + 0.008*"scientist" + 0.007*"go" + 0.007*"build" + 0.007*"game" + 0.006*"ipad" + 0.005*"internet" 
Words: 3


Topic: 0.010*"tech" + 0.007*"app" + 0.007*"microsoft" + 0.007*"compani" + 0.007*"game" + 0.007*"chang" + 0.007*"googl" + 0.007*"climat" + 0.007*"wait" + 0.007*"week" 
Words: 4


Topic: 0.011*"game" + 0.009*"build" + 0.009*"window" + 0.009*"meet" + 0.008*"turn" + 0.007*"futur" +

Running LDA using TF IDF

In [ ]:
lda_model_tfidf_Reports = gensim.models.LdaMulticore(corpus_tfidf_Reports, 
                                       num_topics=10, 
                                       id2word = dictionary_Reports, 
                                       passes = 2, 
                                       workers=2)

for idx, topic in lda_model_tfidf_Reports.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.006*"million" + 0.005*"game" + 0.005*"build" + 0.005*"kill" + 0.005*"life" + 0.005*"android" + 0.005*"give" + 0.004*"facebook" + 0.004*"leak" + 0.004*"year" 
Words: 0


Topic: 0.006*"mar" + 0.005*"twitter" + 0.004*"latest" + 0.004*"electr" + 0.004*"microsoft" + 0.004*"test" + 0.004*"drive" + 0.003*"wait" + 0.003*"week" + 0.003*"readi" 
Words: 1


Topic: 0.007*"samsung" + 0.006*"world" + 0.004*"phone" + 0.004*"great" + 0.004*"realiti" + 0.004*"mysteri" + 0.003*"ballmer" + 0.003*"ferguson" + 0.003*"polic" + 0.003*"surpris" 
Words: 2


Topic: 0.006*"like" + 0.005*"game" + 0.005*"facebook" + 0.004*"best" + 0.004*"look" + 0.004*"internet" + 0.004*"amazon" + 0.004*"show" + 0.003*"launch" + 0.003*"microsoft" 
Words: 3


Topic: 0.004*"neutral" + 0.004*"track" + 0.004*"opiat" + 0.004*"year" + 0.004*"state" + 0.004*"get" + 0.004*"mummi" + 0.004*"war" + 0.003*"life" + 0.003*"hyperloop" 
Words: 4


Topic: 0.005*"friend" + 0.005*"save" + 0.005*"expens" + 0.005*"touch" + 0.005*"appl" + 0.00

In [ ]:
lda_model_tfidf_Reports.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/LDA_TFID_reports.sav')
lda_model_Reports.save('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/lda_model_reports.sav')

In [ ]:
unseen_document = "Romance Between Business and the Republican Party Hits the Rocks"

# Data preprocessing step for the unseen document
bow_vector = dictionary_Reports.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_Reports.print_topic(index, 5)))

Score: 0.3847419321537018	 Topic: 0.008*"googl" + 0.008*"internet" + 0.008*"game" + 0.006*"high" + 0.006*"help"
Score: 0.2552696168422699	 Topic: 0.012*"year" + 0.011*"microsoft" + 0.011*"reveal" + 0.009*"love" + 0.008*"scientist"
Score: 0.21997831761837006	 Topic: 0.010*"life" + 0.010*"phone" + 0.008*"come" + 0.008*"game" + 0.008*"million"
Score: 0.02000642754137516	 Topic: 0.029*"appl" + 0.013*"googl" + 0.010*"want" + 0.010*"world" + 0.009*"robot"
Score: 0.020001830533146858	 Topic: 0.010*"futur" + 0.008*"realiti" + 0.008*"anim" + 0.008*"googl" + 0.008*"virtual"
Score: 0.020001830533146858	 Topic: 0.011*"game" + 0.009*"build" + 0.009*"window" + 0.009*"meet" + 0.008*"turn"
Score: 0.019999999552965164	 Topic: 0.014*"facebook" + 0.011*"look" + 0.010*"like" + 0.010*"world" + 0.009*"twitter"
Score: 0.019999999552965164	 Topic: 0.010*"tech" + 0.007*"app" + 0.007*"microsoft" + 0.007*"compani" + 0.007*"game"
Score: 0.019999999552965164	 Topic: 0.021*"game" + 0.010*"mar" + 0.009*"want" + 0.00

##Test MicroFactor Models

In [ ]:
factors = ['business', 'Reports', 'Book Reviews', 'broadcast']
test = test[test.category.isin(factors)]

In [ ]:
test['category'].unique()

array(['Reports', 'Book Reviews', 'broadcast', 'business'], dtype=object)

In [ ]:
del test['author']
del test['content']
del test['year']
del test['month']
del test['digital']
del test['section']
del test['url']

In [ ]:
del test['date']

In [ ]:
test.head(5)

,id,title,publication,category
137,6426,Breaking down Apple’s new augmented reality pl...,Verge,Reports
143,6432,Could the US stop a nuclear missile attack?,Verge,Reports
147,6436,Hyperloop One wants to help your self-driving ...,Verge,Reports
148,6437,What’s standing between Donald Trump and nucle...,Verge,Reports
150,6439,Giving robots ‘personhood’ is actually about m...,Verge,Reports


In [ ]:
test['business_factor_score'] = 0
# test['business_factor_topic'] = "NAN"
# test['Reports_score'] = ""
# test['broadcast_score'] = ""
# test['BookReviews_score'] = ""
factor_score = []

In [ ]:
for index, row in test.iterrows():
    title = row['title']
    # print(title)
    bow_vector = dictionary_business.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]): 
      row['business_factor_score'] = score
      factor_score.append(score)
      row['business_factor_topic'] = lda_model_business.print_topic(index, 5)
      break;
      

In [ ]:
test['business_factor_score'] = factor_score

In [ ]:
broadcast_score = []

In [ ]:
for index, row in test.iterrows():
    title = row['title']
    bow_vector = dictionary_broadcast.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['broadcast_score'] = score
      broadcast_score.append(score)
      # print(row['broadcast_factor_score'])
      break;

In [ ]:
test['broadcast_score'] = 0

In [ ]:
test['broadcast_score'] = broadcast_score

In [ ]:
BookReviews_score=[]
test['BookReviews_score'] = 0

In [ ]:
for index, row in test.iterrows():
    title = row['title']
    bow_vector = dictionary_BookReviews.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['BookReviews_score'] = score
      BookReviews_score.append(score)
      break;

In [ ]:
test['BookReviews_score'] = BookReviews_score

In [ ]:
report_factor_score=[]
test['report_factor_score'] = 0

In [ ]:
for index, row in test.iterrows():
    title = row['title']
    bow_vector = dictionary_Reports.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['report_factor_score'] = score
      # row['report_factor_topic'] = lda_model_Reports.print_topic(index, 5)
      report_factor_score.append(score)
      break;

In [ ]:
test['report_factor_score'] = report_factor_score

In [ ]:
test['category'].unique()

array(['Reports', 'Book Reviews', 'broadcast', 'business'], dtype=object)

In [ ]:
test.head()

,id,title,publication,category,business_factor_score,broadcast_score,BookReviews_score,report_factor_score
137,6426,Breaking down Apple’s new augmented reality pl...,Verge,Reports,0.274567,0.600801,0.10,0.478131
143,6432,Could the US stop a nuclear missile attack?,Verge,Reports,0.477333,0.626893,0.10,0.406685
147,6436,Hyperloop One wants to help your self-driving ...,Verge,Reports,0.458418,0.390440,0.10,0.740001
148,6437,What’s standing between Donald Trump and nucle...,Verge,Reports,0.458191,0.527997,0.10,0.274998
150,6439,Giving robots ‘personhood’ is actually about m...,Verge,Reports,0.460901,0.374496,0.55,0.474098


In [ ]:
error=0

In [ ]:
for index, row in test.iterrows():
  if row['category']=="Reports":
    error+=(1-row['report_factor_score'])
  elif row['category']=="Book Reviews":
    error+=(1-row['BookReviews_score'])
  elif row['category']=="broadcast":
    error+=(1-row['broadcast_score'])
  elif row['category']=="business":
    error+=(1-row['business_factor_score'])


RMSE of the model

In [ ]:
print("RMSE:",np.sqrt(error/test.shape[0]))

RMSE: 0.6581001401170543


##Load new data and get the dataset to be classified into different microfactors

In [ ]:
test_filename = '/content/drive/MyDrive/ML-Spring-2022/AlternusVera/test.tsv'
train_filename = '/content/drive/MyDrive/ML-Spring-2022/AlternusVera/train.tsv'
valid_filename = '/content/drive/MyDrive/ML-Spring-2022/AlternusVera/valid.tsv'

colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

train_news = pd.read_csv(train_filename, sep='\t', names = colnames, error_bad_lines=False)
test_news = pd.read_csv(test_filename, sep='\t', names = colnames, error_bad_lines=False)
valid_news = pd.read_csv(valid_filename, sep='\t', names = colnames, error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [ ]:
# del train_news['speaker']
# del train_news['speakerjobtitle']
# del train_news['stateinfo']
# del train_news['partyaffiliation']
# del train_news['barelytruecounts']
# del train_news['falsecounts']
# del train_news['halftruecounts']
# del train_news['mostlytrueocunts']
# del train_news['pantsonfirecounts']

In [ ]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [ ]:
train_news.shape

(10240, 14)

##Business Score to New Data

Adding topic and score of each micro factor to text in dataframe

In [ ]:
business_factor_score = []
train_news['business_factor_score'] = " "

In [ ]:
for index, row in train_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_business.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['business_factor_score'] = score
      business_factor_score.append(score)
      # row['business_factor_topic'] = lda_model_business.print_topic(index, 5)
      break;


In [ ]:
train_news['business_factor_score'] = business_factor_score

In [ ]:
train_news

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,business_factor_score
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0.703164
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0.396987
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,0.605577
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0.317883
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0.436707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",0.349660
10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,0.518351
10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,0.370382
10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,0.667827


In [ ]:
business_factor_score1 = []
test_news['business_factor_score'] = " "

for index, row in test_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_business.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['business_factor_score'] = score
      business_factor_score1.append(score)
      # row['business_factor_topic'] = lda_model_business.print_topic(index, 5)
      break;

test_news['business_factor_score'] = business_factor_score1

##Broadcast Factor score on New Data

In [ ]:
broadcast_factor_score=[]
train_news['broadcast_factor_score'] = ""

In [ ]:
for index, row in train_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_broadcast.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['broadcast_factor_score'] = score
      broadcast_factor_score.append(score)
      # row['broadcast_factor_topic'] = lda_model_broadcast.print_topic(index, 5)
      # print(row['broadcast_factor_score'])
      break;

In [ ]:
train_news['broadcast_factor_score'] = broadcast_factor_score

In [ ]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,business_factor_score,broadcast_factor_score
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0.703164,0.549374
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0.396987,0.846397
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,0.605577,0.438088
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0.317883,0.403089
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0.436707,0.532862


In [ ]:
broadcast_factor_score1=[]
test_news['broadcast_factor_score'] = ""

for index, row in test_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_broadcast.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['broadcast_factor_score'] = score
      broadcast_factor_score1.append(score)
      # row['broadcast_factor_topic'] = lda_model_broadcast.print_topic(index, 5)
      # print(row['broadcast_factor_score'])
      break;

test_news['broadcast_factor_score'] = broadcast_factor_score1

##Book Review factor score for new Data

In [ ]:
bookReview_factor_score=[]
train_news['bookReview_factor_score'] = ""

In [ ]:
for index, row in train_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_BookReviews.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['bookReview_factor_score'] = score
      bookReview_factor_score.append(score)
      # row['bookReview_factor_topic'] = lda_model_BookReviews.print_topic(index, 5)
      break;

In [ ]:
train_news['bookReview_factor_score'] = bookReview_factor_score

In [ ]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,business_factor_score,broadcast_factor_score,bookReview_factor_score
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0.703164,0.549374,0.100000
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0.396987,0.846397,0.100000
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,0.605577,0.438088,0.100000
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0.317883,0.403089,0.549979
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0.436707,0.532862,0.100000


In [ ]:
bookReview_factor_score1=[]
test_news['bookReview_factor_score'] = ""

for index, row in test_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_BookReviews.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['bookReview_factor_score'] = score
      bookReview_factor_score1.append(score)
      # row['bookReview_factor_topic'] = lda_model_BookReviews.print_topic(index, 5)
      break;
  
test_news['bookReview_factor_score'] = bookReview_factor_score1

## Report factor score for new data

In [ ]:
report_factor_score = []
train_news['report_factor_score'] = ""

In [ ]:
for index, row in train_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_Reports.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['report_factor_score'] = score
      report_factor_score.append(score)
      # row['report_factor_topic'] = lda_model_Reports.print_topic(index, 5)
      break;

In [ ]:
train_news['report_factor_score'] =report_factor_score

In [ ]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,business_factor_score,broadcast_factor_score,bookReview_factor_score,report_factor_score
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0.703164,0.549374,0.100000,0.350016
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0.396987,0.846397,0.100000,0.481591
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,0.605577,0.438088,0.100000,0.366671
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0.317883,0.403089,0.549979,0.325220
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0.436707,0.532862,0.100000,0.524995


In [ ]:
report_factor_score1 = []
test_news['report_factor_score'] = ""

for index, row in test_news.iterrows():
    title = row['headline_text']
    bow_vector = dictionary_Reports.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
      # row['report_factor_score'] = score
      report_factor_score1.append(score)
      # row['report_factor_topic'] = lda_model_Reports.print_topic(index, 5)
      break;

test_news['report_factor_score'] =report_factor_score1

Save dataset to drive

In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Sprint3NLP/trainFactorScore.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  train_news.to_csv(f)

save test dataset

In [ ]:
path = '/content/drive/MyDrive/ML-Spring-2022/TheTrio/Sprint3NLP/testFactorScore.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  test_news.to_csv(f)

So above dataframe gives the score for microfactors for given headline text.   
The score for each microfactor gives microfactor coverage in the text

In [149]:
train_news.head(5)

,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context,business_factor_score,broadcast_factor_score,bookReview_factor_score,report_factor_score
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0.703164,0.549374,0.100000,0.350016
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0.396987,0.846397,0.100000,0.481591
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,0.605577,0.438088,0.100000,0.366671
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0.317883,0.403089,0.549979,0.325220
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0.436707,0.532862,0.100000,0.524995


##save model

In [138]:
class NewsCoverage:
  @classmethod
  def get_BookReview_score(self, text):
    score_business=0
    bow_vector = dictionary_BookReviews.doc2bow(preprocess(text))
    for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]):
      score_business = score
      break;
    return score_business

  @classmethod
  def get_Report_score(self, text):
    score_report=0
    bow_vector = dictionary_Reports.doc2bow(preprocess(text))
    for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
      score_report=score
      break;
    return score_report

  @classmethod
  def get_business_score(self, text):
    score_business=0
    bow_vector = dictionary_business.doc2bow(preprocess(title))
    for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]):
      score_business = score
      break;
    return score_business

  @classmethod
  def get_Broadcast_score(self, text):
    score_broadcast=0
    bow_vector = dictionary_broadcast.doc2bow(preprocess(text))
    for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
      score_broadcast = score
      break;
    return score_broadcast

  @classmethod
  def getAvergaescore(self,text):

    average_score = (self.get_BookReview_score(text) + self.get_Report_score(text) + self.get_business_score(text) + self.get_Broadcast_score(text))/4
    return average_score
  



##pickle model

In [143]:
import pickle

NewsCoverageModel = NewsCoverage()
NewsCoverageModel_pickle = pickle.dumps(NewsCoverageModel)
with open('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Sprint3NLP/NewsCoverageModelPickle.pickle', 'wb') as f:
    pickle.dump(NewsCoverageModel, f)

In [145]:
unseen_document = "Country music legend Naomi Judd, one half of The Judds, dies age 76"


In [147]:
pickel_model = pickle.load(open('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Sprint3NLP/NewsCoverageModelPickle.pickle', 'rb'))
print(pickel_model.get_Broadcast_score(unseen_document))

0.7040343


#Scrape some article and get score for each microfactor

In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import nltk
from urllib.request import urlopen
import json
import logging
import re

In [ ]:
def beautify_txt(text):
    final_data = (((text).replace(u'\xa0', u' ')).replace(r'\r',u'\n').replace(r'\n', ' '))
    return text

articles=["https://www.vashonhistory.com/Vashon%20History/Newspaaper/newspaper_1937.htm"
          # "https://www.vashonhistory.com/Vashon%20History/Newspaaper/newspaper_1936.htm",
          # "https://www.vashonhistory.com/Vashon%20History/Newspaaper/newspaper_1935.htm",
          # "https://www.vashonhistory.com/Vashon%20History/Newspaaper/newspaper_1934.htm"
          ]

articalName = ["article1", "article2","article3", "article4"]

In [ ]:
nltk.download('punkt')

for x in range(len(articles)):
     p_string = ""
     html_source = urlopen(articles[x])

     soup = BeautifulSoup(html_source, 'html.parser')
     try:
      for div in soup.find_all('section', class_='section'):
          for li in div.find_all('li'):
              p_tag = li.find('p')
              p_string = p_string + p_tag.getText()
              print(p_tag.getText())
      
     except Exception as e:
        logging.error(e)
      
     with open('/content/drive/MyDrive/ML-Spring-2022/TheTrio/Harika/NLP-Assignments/Assignment-10-AlternusVera_Part1&2/dataset/' + articalName[x] + '.txt', 'w') as outfile:
        json.dump(p_string, outfile)

In [ ]:
p_string

Reports score

In [ ]:
article = p_string

# Data preprocessing step for the unseen document
bow_vector = dictionary_Reports.doc2bow(preprocess(article))

for index, score in sorted(lda_model_Reports[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_Reports.print_topic(index, 5)))

Score: 0.14730697870254517	 Topic: 0.014*"facebook" + 0.011*"look" + 0.010*"like" + 0.010*"world" + 0.009*"twitter"
Score: 0.13102518022060394	 Topic: 0.029*"appl" + 0.013*"googl" + 0.010*"want" + 0.010*"world" + 0.009*"robot"
Score: 0.1275797337293625	 Topic: 0.011*"game" + 0.009*"build" + 0.009*"window" + 0.009*"meet" + 0.008*"turn"
Score: 0.11259030550718307	 Topic: 0.012*"year" + 0.011*"microsoft" + 0.011*"reveal" + 0.009*"love" + 0.008*"scientist"
Score: 0.10357066243886948	 Topic: 0.008*"googl" + 0.008*"internet" + 0.008*"game" + 0.006*"high" + 0.006*"help"
Score: 0.08147517591714859	 Topic: 0.010*"tech" + 0.007*"app" + 0.007*"microsoft" + 0.007*"compani" + 0.007*"game"
Score: 0.07823462784290314	 Topic: 0.010*"life" + 0.010*"phone" + 0.008*"come" + 0.008*"game" + 0.008*"million"
Score: 0.07774028182029724	 Topic: 0.012*"futur" + 0.010*"problem" + 0.010*"stream" + 0.005*"android" + 0.005*"time"
Score: 0.07727862149477005	 Topic: 0.021*"game" + 0.010*"mar" + 0.009*"want" + 0.008*"

Book Review score

In [ ]:
article = p_string

# Data preprocessing step for the unseen document
bow_vector = dictionary_BookReviews.doc2bow(preprocess(article))

for index, score in sorted(lda_model_BookReviews[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_BookReviews.print_topic(index, 5)))

Score: 0.23473091423511505	 Topic: 0.052*"chang" + 0.052*"robinson" + 0.052*"think" + 0.052*"experi" + 0.052*"glorious"
Score: 0.20968692004680634	 Topic: 0.070*"world" + 0.037*"guid" + 0.037*"atlas" + 0.037*"think" + 0.037*"beat"
Score: 0.19335398077964783	 Topic: 0.064*"book" + 0.064*"futur" + 0.064*"form" + 0.064*"data" + 0.064*"gibson"
Score: 0.16822032630443573	 Topic: 0.058*"book" + 0.058*"mar" + 0.058*"terribl" + 0.058*"cost" + 0.058*"wander"
Score: 0.11451715230941772	 Topic: 0.078*"malka" + 0.078*"infomocraci" + 0.078*"thriller" + 0.078*"season" + 0.078*"essenti"
Score: 0.05050560459494591	 Topic: 0.091*"book" + 0.091*"page" + 0.091*"fiction" + 0.091*"scienc" + 0.091*"histori"
Score: 0.02805919758975506	 Topic: 0.073*"novel" + 0.073*"lack" + 0.073*"dinosaur" + 0.073*"gunsling" + 0.073*"bone"


Broadcast

In [ ]:
article = p_string

# Data preprocessing step for the unseen document
bow_vector = dictionary_broadcast.doc2bow(preprocess(article))

for index, score in sorted(lda_model_broadcast[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_broadcast.print_topic(index, 5)))

Score: 0.15106000006198883	 Topic: 0.030*"trump" + 0.030*"clinton" + 0.015*"video" + 0.009*"email" + 0.008*"poll"
Score: 0.1298665702342987	 Topic: 0.023*"trump" + 0.022*"fact" + 0.022*"fast" + 0.006*"attack" + 0.006*"time"
Score: 0.10118021070957184	 Topic: 0.035*"trump" + 0.011*"shoot" + 0.010*"video" + 0.010*"clinton" + 0.008*"obama"
Score: 0.10025236010551453	 Topic: 0.022*"trump" + 0.021*"kill" + 0.010*"say" + 0.010*"student" + 0.007*"isi"
Score: 0.09973261505365372	 Topic: 0.038*"video" + 0.015*"trump" + 0.006*"obama" + 0.006*"teen" + 0.006*"near"
Score: 0.09444732964038849	 Topic: 0.027*"trump" + 0.018*"say" + 0.013*"clinton" + 0.006*"hous" + 0.006*"report"
Score: 0.09292024374008179	 Topic: 0.048*"trump" + 0.015*"clinton" + 0.014*"obama" + 0.010*"korea" + 0.010*"north"
Score: 0.08884128928184509	 Topic: 0.039*"trump" + 0.014*"world" + 0.008*"look" + 0.006*"polici" + 0.006*"kill"
Score: 0.07806207984685898	 Topic: 0.013*"shoot" + 0.013*"attack" + 0.013*"polic" + 0.011*"dead" + 0

Business

In [ ]:
article = p_string

# Data preprocessing step for the unseen document
bow_vector = dictionary_business.doc2bow(preprocess(article))

for index, score in sorted(lda_model_business[bow_vector], key=lambda tup: -1*tup[1]): 
    print("Score: {}\t Topic: {}".format(score, lda_model_business.print_topic(index, 5)))

Score: 0.13403108716011047	 Topic: 0.030*"trump" + 0.010*"go" + 0.010*"best" + 0.010*"googl" + 0.010*"latest"
Score: 0.1280624121427536	 Topic: 0.037*"trump" + 0.015*"donald" + 0.008*"clinton" + 0.007*"make" + 0.007*"hillari"
Score: 0.1184440329670906	 Topic: 0.016*"trump" + 0.014*"hous" + 0.011*"appl" + 0.010*"report" + 0.007*"white"
Score: 0.11004222929477692	 Topic: 0.042*"trump" + 0.014*"donald" + 0.009*"clinton" + 0.009*"obama" + 0.008*"hillari"
Score: 0.09836255759000778	 Topic: 0.020*"year" + 0.017*"trump" + 0.016*"come" + 0.011*"protect" + 0.011*"type"
Score: 0.0935051441192627	 Topic: 0.011*"trump" + 0.007*"like" + 0.007*"clinton" + 0.006*"want" + 0.006*"biggest"
Score: 0.08933085203170776	 Topic: 0.012*"trump" + 0.011*"wall" + 0.011*"street" + 0.009*"like" + 0.008*"explain"
Score: 0.086916983127594	 Topic: 0.037*"trump" + 0.014*"say" + 0.011*"compani" + 0.009*"world" + 0.009*"help"
Score: 0.08357729762792587	 Topic: 0.014*"tech" + 0.013*"happen" + 0.013*"high" + 0.012*"thing"

Conclusion:
The article has more news covergae related to book reviews